# Déploiement du modèle sur Azure Machine Learning

(Ce notebook est à utiliser sur Azure Machine Learning)

On va déployer le modèle entraîné et sauvegardé comme un web service sur Azure Container Instances (ACI)

## Récupération du modèle

In [41]:
# Récupérer le modèle entraîné
from azureml.core import Workspace
ws = Workspace.from_config()

from azureml.core.model import Model

model_name = "bidirectional_lstm"
model = Model.register(model_path="outputs/bidirectional_lstm.pkl",
                        model_name=model_name,
                        tags={"area": "NLP", "model": "deep_learning classification"},
                        description="Modèle keras bidirectionnal lstm entraîné",
                        workspace=ws)

Registering model bidirectional_lstm


## Créer le script score

The script de scoring "score.py", est utilisé par lors de l'appel du web service pour utiliser le modèle.

Il est composé de 2 fonctions :

- **init()** : qui charge le modèle dans un objet globam. Cette fonction est appelée une fois quand le container Docker est démarré.
- **run(input_data)** : fonction qui utilise le modèle pour prédire le résultat à partir des données d'entrée.

In [42]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path(model_name='bidirectional_lstm')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_pred_proba = model.predict(data)
    # you can return any data type as long as it is JSON-serializable

    y_pred = [round(pred_proba[0]) for pred_proba in y_pred_proba.tolist()]

    return y_pred_proba.tolist(), y_pred

Overwriting score.py


## Fichier de configuration

Créer un fichier de configuration de déploiement, spécifier le nombre de CPUs et de RAM à utiliser pour le container.

In [43]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 4, auth_enabled=True)

## Création de l'environnement et de l'inférence

On spécifie les dépendances nécessaires pour l'environnement

In [44]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

env = Environment('projet_7')

cd = CondaDependencies.create(pip_packages=['scikit-learn' ,'keras' ,'tensorflow', 'azureml-defaults'])

env.python.conda_dependencies = cd


inference_config = InferenceConfig(entry_script="score.py",
                                   environment=env)

## Déploiement du modèle

In [45]:
service = Model.deploy(workspace=ws, 
                       name='tweets-sentiment-analysis', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-06-13 16:49:12+00:00 Creating Container Registry if not exists.
2022-06-13 16:49:12+00:00 Registering the environment.
2022-06-13 16:49:13+00:00 Use the existing image.
2022-06-13 16:49:13+00:00 Generating deployment configuration.
2022-06-13 16:49:14+00:00 Submitting deployment to compute.
2022-06-13 16:49:20+00:00 Checking the status of deployment tweets-sentiment-analysis..
2022-06-13 16:51:28+00:00 Checking the status of inference endpoint tweets-sentiment-analysis.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [46]:
primary, secondary = service.get_keys()
print(primary)
print(service.scoring_uri)

m4zJCjMjthcFYsRacsTUBS4RN3xU7lJp
http://8b19886d-a159-4585-a9a4-975a4924c7d7.westeurope.azurecontainer.io/score


## Test du modèle en appelant le Web Service

In [50]:
import json

input_payload = json.dumps({
    'data': ["I am having a very good day today", "I hate when it's raining outside", "My car is very beautiful"]
})
output = service.run(input_payload)

print(output)

[[[0.9965319037437439], [0.004983492195606232], [0.9657612442970276]], [1, 0, 1]]


## Suppression du Web Service

In [40]:
service.delete()